# IBM Data Science Professional Certification - Capstone Project
### This notebook will be mainly used for the completion of the certification's last course

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [2]:
# Downloading the dataframe
!wget -O Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

--2020-09-30 22:09:21--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘Collisions.csv’

100%[======================================>] 73,917,638  42.9MB/s   in 1.6s   

2020-09-30 22:09:23 (42.9 MB/s) - ‘Collisions.csv’ saved [73917638/73917638]



In [3]:
# Loading the Dataset
df = pd.read_csv('Collisions.csv')
df.head()
print('Initial shape of Dataframe:', df.shape)
df.SEVERITYDESC.value_counts()

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Initial shape of Dataframe: (194673, 38)


Property Damage Only Collision    136485
Injury Collision                   58188
Name: SEVERITYDESC, dtype: int64

# Feature selection

<font size=2.5>After a first looking at the description of the columns provided in the metadata file, I performed this initial feature selection:</font>

In [4]:
# features important for prediction:
features = df[['X', 'Y', 'ADDRTYPE','SDOT_COLCODE', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT',
               'PEDCYLCOUNT', 'VEHCOUNT', 'INATTENTIONIND', 'UNDERINFL','WEATHER', 'ROADCOND', 'LIGHTCOND','SPEEDING', 'SEVERITYCODE']]
features.head()

,X,Y,ADDRTYPE,SDOT_COLCODE,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,SPEEDING,SEVERITYCODE
0,-122.323148,47.703140,Intersection,11,Angles,2,0,0,2,NaN,N,Overcast,Wet,Daylight,NaN,2
1,-122.347294,47.647172,Block,16,Sideswipe,2,0,0,2,NaN,0,Raining,Wet,Dark - Street Lights On,NaN,1
2,-122.334540,47.607871,Block,14,Parked Car,4,0,0,3,NaN,0,Overcast,Dry,Daylight,NaN,1
3,-122.334803,47.604803,Block,11,Other,3,0,0,3,NaN,N,Clear,Dry,Daylight,NaN,1
4,-122.306426,47.545739,Intersection,11,Angles,2,0,0,2,NaN,0,Raining,Wet,Daylight,NaN,2


<font size=2.5>Looking at the inputs of the "SPEEDING" and "INATTENTIONIND" columns, we see that most of the rows are missing values, so we remove these columns from our features dataframe.</font>

In [5]:
print(features['SPEEDING'].value_counts())
print(features['INATTENTIONIND'].value_counts())
print('\n')
features.drop(['SPEEDING', 'INATTENTIONIND'], axis=1, inplace=True)
features.info()

Y    9333
Name: SPEEDING, dtype: int64
Y    29805
Name: INATTENTIONIND, dtype: int64


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194673 entries, 0 to 194672
Data columns (total 14 columns):
X                189339 non-null float64
Y                189339 non-null float64
ADDRTYPE         192747 non-null object
SDOT_COLCODE     194673 non-null int64
COLLISIONTYPE    189769 non-null object
PERSONCOUNT      194673 non-null int64
PEDCOUNT         194673 non-null int64
PEDCYLCOUNT      194673 non-null int64
VEHCOUNT         194673 non-null int64
UNDERINFL        189789 non-null object
WEATHER          189592 non-null object
ROADCOND         189661 non-null object
LIGHTCOND        189503 non-null object
SEVERITYCODE     194673 non-null int64
dtypes: float64(2), int64(6), object(6)
memory usage: 20.8+ MB


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# Data preprocessing

<font size=2.5>Removing the missing entries:</font>

In [6]:
features.dropna(inplace = True)
features.reset_index()
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184146 entries, 0 to 194672
Data columns (total 14 columns):
X                184146 non-null float64
Y                184146 non-null float64
ADDRTYPE         184146 non-null object
SDOT_COLCODE     184146 non-null int64
COLLISIONTYPE    184146 non-null object
PERSONCOUNT      184146 non-null int64
PEDCOUNT         184146 non-null int64
PEDCYLCOUNT      184146 non-null int64
VEHCOUNT         184146 non-null int64
UNDERINFL        184146 non-null object
WEATHER          184146 non-null object
ROADCOND         184146 non-null object
LIGHTCOND        184146 non-null object
SEVERITYCODE     184146 non-null int64
dtypes: float64(2), int64(6), object(6)
memory usage: 21.1+ MB


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


<font size=2.5>If we analyse the values present in the "UNDERINFL" columns we see that there are different labels for the same information. Let's standardise the labels:</font>

In [7]:
print(features['UNDERINFL'].value_counts())
# I created this function in order to unify the codification fo the values
def standardize_format(val):
    if val == 'N':
        return '0'
    elif val == 'Y':
        return '1'
    else:
        return val
    
features['UNDERINFL'] = features['UNDERINFL'].apply(standardize_format)
features['UNDERINFL'] = features['UNDERINFL'].astype(int)
features['UNDERINFL'].value_counts()

N    97224
0    78064
Y     4979
1     3879
Name: UNDERINFL, dtype: int64


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    175288
1      8858
Name: UNDERINFL, dtype: int64

<font size=2.5>Let's convert the categorical variables into numerical format using One Hot encoding</font>

In [8]:
encoded_features = pd.get_dummies(features, columns=['ADDRTYPE', 'COLLISIONTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND'], dtype='int64')
encoded_features.info()
print('\n')
encoded_features.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184146 entries, 0 to 194672
Data columns (total 50 columns):
X                                     184146 non-null float64
Y                                     184146 non-null float64
SDOT_COLCODE                          184146 non-null int64
PERSONCOUNT                           184146 non-null int64
PEDCOUNT                              184146 non-null int64
PEDCYLCOUNT                           184146 non-null int64
VEHCOUNT                              184146 non-null int64
UNDERINFL                             184146 non-null int64
SEVERITYCODE                          184146 non-null int64
ADDRTYPE_Block                        184146 non-null int64
ADDRTYPE_Intersection                 184146 non-null int64
COLLISIONTYPE_Angles                  184146 non-null int64
COLLISIONTYPE_Cycles                  184146 non-null int64
COLLISIONTYPE_Head On                 184146 non-null int64
COLLISIONTYPE_Left Turn               184146 

,X,Y,SDOT_COLCODE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,UNDERINFL,SEVERITYCODE,ADDRTYPE_Block,...,ROADCOND_Wet,LIGHTCOND_Dark - No Street Lights,LIGHTCOND_Dark - Street Lights Off,LIGHTCOND_Dark - Street Lights On,LIGHTCOND_Dark - Unknown Lighting,LIGHTCOND_Dawn,LIGHTCOND_Daylight,LIGHTCOND_Dusk,LIGHTCOND_Other,LIGHTCOND_Unknown
count,184146.000000,184146.000000,184146.000000,184146.000000,184146.000000,184146.000000,184146.000000,184146.000000,184146.000000,184146.000000,...,184146.000000,184146.000000,184146.000000,184146.000000,184146.000000,184146.000000,184146.000000,184146.000000,184146.000000,184146.000000
mean,-122.330501,47.619546,14.041348,2.468085,0.038850,0.029661,1.973782,0.048103,1.304107,0.657180,...,0.249796,0.007869,0.006218,0.256574,0.000060,0.013147,0.616408,0.031290,0.001021,0.067414
std,0.030018,0.056216,6.702767,1.358666,0.202567,0.170991,0.556125,0.213985,0.460029,0.474653,...,0.432896,0.088357,0.078608,0.436743,0.007729,0.113905,0.486262,0.174102,0.031936,0.250738
min,-122.419091,47.495573,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-122.348839,47.575884,11.000000,2.000000,0.000000,0.000000,2.000000,0.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,-122.330168,47.615361,13.000000,2.000000,0.000000,0.000000,2.000000,0.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,-122.311922,47.663835,14.000000,3.000000,0.000000,0.000000,2.000000,0.000000,2.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,-122.238949,47.734142,69.000000,81.000000,6.000000,2.000000,12.000000,1.000000,2.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


<font size=2.5>Let's scale the continuous columns to allow the model to be trained faster:</font> 

In [9]:
from sklearn.preprocessing import StandardScaler
encoded_features[['X', 'Y']] = StandardScaler().fit_transform(encoded_features[['X', 'Y']])
encoded_features.head()

,X,Y,SDOT_COLCODE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,UNDERINFL,SEVERITYCODE,ADDRTYPE_Block,...,ROADCOND_Wet,LIGHTCOND_Dark - No Street Lights,LIGHTCOND_Dark - Street Lights Off,LIGHTCOND_Dark - Street Lights On,LIGHTCOND_Dark - Unknown Lighting,LIGHTCOND_Dawn,LIGHTCOND_Daylight,LIGHTCOND_Dusk,LIGHTCOND_Other,LIGHTCOND_Unknown
0,0.244935,1.487029,11,2,0,0,2,0,2,0,...,1,0,0,0,0,0,1,0,0,0
1,-0.559436,0.491441,16,2,0,0,2,0,1,1,...,1,0,0,1,0,0,0,0,0,0
2,-0.134553,-0.207669,14,4,0,0,3,0,1,1,...,0,0,0,0,0,0,1,0,0,0
3,-0.143324,-0.262254,11,3,0,0,3,0,1,1,...,0,0,0,0,0,0,1,0,0,0
4,0.801998,-1.312909,11,2,0,0,2,0,2,0,...,1,0,0,0,0,0,1,0,0,0


In [10]:
# now lets finally get the matrix of features an the target vector
X = encoded_features.drop('SEVERITYCODE', axis=1)
y = encoded_features['SEVERITYCODE']
print(X.shape)
print(y.shape)

(184146, 49)
(184146,)


## Creating the model

<font size=2.5>Now that we have the preprocessed the dataset, let's fit it to a model. I decided to use a XGBClassifier, which is an ensemble machine learning model. This means that it is comprised of several models(yes! models within the model hehe) that are called base learners, which have an accuracy slightly better than chance. The output of each of these models count as a vote. This votes are counted and the majority is the final output. For more information about XGBClassifier click the following link: <a href='https://xgboost.readthedocs.io/en/latest/python/python_api.html'>xgboost documentation</a>. <font>

In [11]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

# splitting the dataset into training an test set. Note that, as y labels are inbalanced, we are going to use the 'stratify' parameter of the train_test_split function in order to mitigate bias
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
# see how the proportion of labels is the same for both the training and test set
print(X_train.shape)
print(X_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())



(147316, 49)
(36830, 49)
1    102516
2     44800
Name: SEVERITYCODE, dtype: int64
1    25630
2    11200
Name: SEVERITYCODE, dtype: int64


In [12]:
# Hyperparamters tunning
# UNCOMMENT IF YOU WANT TO FINE TUNE THE MODEL. IT MAY TAKE A WHILE DEPENDING ON YOUR COMPUTONG POWER!!!
# from sklearn.model_selection import GridSearchCV
# param_grid = {'learning_rate': [0.01,0.1, 1, 10],'n_estimators': [100,200, 300, 400, 500], 'subsample':[ 0.9, 1], 'max_depth': [3, 5, 7, 9], 'reg_lambda': [0.01, 0.1, 1, 10], 'reg_alpha': [0.01, 0.1, 1, 10]}
# grid_f1 = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring='f1', cv=5, verbose=10)
# grid_f1.fit(X_train, y_train)
# print("Best parameters found: ",grid_f1.best_params_)
# print("best score: ", grid_f1.best_score_)

<font size=2.5>After searching for the best parameters in my personal computer, I will build the model with the best parameters found</font>

In [13]:
from sklearn.metrics import f1_score
classifier = xgb.XGBClassifier(learning_rate=0.01, n_estimators=200,subsample=0.9, max_depth=5, reg_lambda=0.01, reg_alpha=0.01)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
#taking a glimpse at the first  5 observations
y_pred[:5]

array([1, 1, 1, 1, 2])

## Evaluating model's performance

In [14]:
from sklearn.metrics import confusion_matrix, f1_score
print('The f1 score of the model is: ', f1_score(y_test, y_pred))
# let's see the strength and weaknesses through a confusion matrix
confusion_matrix(y_test, y_pred)

The f1 score of the model is:  0.8469841485353519


array([[24953,   677],
       [ 8339,  2861]])

<font size=2.5>Our model got a f1-score of approximately 84% in the test set, even though it have never seen the data before. This is a good result for a prototyping solution. Further steps to increase the score could be consider text features like ST_COLDESC that provide a textual description of the accident by the Police officers,what would involve some natural language processing skills.</font>